In [ ]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager

def setup_driver():
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')  # Enable headless mode if GUI is not needed
    driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
    return driver

def extract_data(driver, url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.category-page-p6sjqa')))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [ ]:
def write_to_csv(soup, filename, attribute_class, parameter):
    total_items = soup.find('div', class_='category-page-p6sjqa').text[:-6]
    elements = soup.find_all('div', class_=attribute_class)
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Parameter", "Parameter Value", "Data Value", "Total Items"])
        for element in elements:
            detail = element.find('input', class_="category-page-40vz4u")
            if detail:
                param_value = detail['value']
                param_id = detail['id']
                data_value = detail.find_next_sibling('span', class_='item_count').text.strip('()')
                writer.writerow([parameter, param_value, data_value, total_items])

In [ ]:
def main():
    driver = setup_driver()
    try:
        url = 'https://oldnavy.gap.com/browse/category.do?cid=1035712&#department=136&facetOrder=department:136'
        soup = extract_data(driver, url)
        write_to_csv(soup, 'old_navy_color_data.csv', 'facetbar-item-colorswatch category-page-5h7d9x', 'Color')
        print("Data extraction and file writing completed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()